In [1]:
from tqdm import tqdm
import numpy as np
import torch
from torch.utils.data import Dataset,DataLoader, SubsetRandomSampler
import config

In [7]:
from datasets.aileen_object import AileenObjectDataset
from model import AttributeNetwork, VGG_Net, RES_Net

In [12]:
shapes = ['cone', 'box', 'cylinder', 'sphere', 'capsule']
sizes = ['small', 'medium', 'large']
colors = ['blue', 'red', 'green', 'yellow', 'purple']

testData = AileenObjectDataset(config.ROOT_TEST)
test_loader = DataLoader(dataset=testData, shuffle=False, batch_size=1)
test_size = len(testData)
model = RES_Net(13)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    model.cuda()

In [13]:
print('Load weights...')
model.load_state_dict(torch.load('attribute_resnet_best_model.pth'))
print('Weights are loaded...')
model.to(device)

Load weights...
Weights are loaded...


RES_Net(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

In [14]:
shape_correct = 0
size_correct = 0
color_correct = 0
with torch.no_grad():
    for x_batch, y_batch in tqdm(test_loader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        y_test_pred = model(x_batch)
        # _, y_pred_tag = torch.max(y_test_pred, dim = 1)
        y_test_pred = y_test_pred.cpu().numpy().copy()
        pred_shape = y_test_pred[0][0:5]
        pred_size = y_test_pred[0][5:8]
        pred_color = y_test_pred[0][8:13]
        flag = np.bool(y_test_pred[0][1])
        index_max = np.where(pred_shape == np.max(pred_shape))[0][0]
        # print pred_shape
        shape = shapes[np.where(pred_shape == np.max(pred_shape))[0][0]]
        size = sizes[np.where(pred_size == max(pred_size))[0][0]]
        color = colors[np.where(pred_color == max(pred_color))[0][0]]

        gt_y = y_batch.cpu().numpy().copy()
        gt_shape = gt_y[0][0:5]
        gt_size = gt_y[0][5:8]
        gt_color = gt_y[0][8:13]
        # print gt_y
        # print np.where(gt_shape == np.max(gt_shape))
        gtshape = shapes[np.where(gt_shape == np.max(gt_shape))[0][0]]
        gtsize = sizes[np.where(gt_size == np.max(gt_size))[0][0]]
        gtcolor = colors[np.where(gt_color == np.max(gt_color))[0][0]]
        if shape == gtshape:
            shape_correct += 1
        if size == gtsize:
            size_correct += 1
        if color == gtcolor:
            color_correct += 1
        print("****************")
        print("predicted shape: %s    gt shape: %s" % (shape, gtshape))
        print("predicted size: %s    gt size: %s" % (size, gtsize))
        print("predicted color: %s    gt color: %s" % (color, gtcolor))
        print("has round surface: %r" % flag)
        print("****************")

  0%|                                         | 11/7676 [00:00<03:12, 39.80it/s]

****************
predicted shape: sphere    gt shape: sphere
predicted size: medium    gt size: medium
predicted color: blue    gt color: blue
has round surface: True
****************
****************
predicted shape: sphere    gt shape: sphere
predicted size: medium    gt size: medium
predicted color: purple    gt color: purple
has round surface: True
****************
****************
predicted shape: box    gt shape: box
predicted size: medium    gt size: medium
predicted color: green    gt color: green
has round surface: True
****************
****************
predicted shape: cone    gt shape: cone
predicted size: medium    gt size: medium
predicted color: yellow    gt color: yellow
has round surface: True
****************
****************
predicted shape: box    gt shape: box
predicted size: large    gt size: large
predicted color: red    gt color: red
has round surface: True
****************
****************
predicted shape: sphere    gt shape: sphere
predicted size: large    gt s

  0%|▏                                        | 29/7676 [00:00<01:55, 66.40it/s]

****************
predicted shape: sphere    gt shape: sphere
predicted size: small    gt size: small
predicted color: yellow    gt color: yellow
has round surface: True
****************
****************
predicted shape: sphere    gt shape: sphere
predicted size: small    gt size: large
predicted color: yellow    gt color: yellow
has round surface: True
****************
****************
predicted shape: box    gt shape: cylinder
predicted size: large    gt size: large
predicted color: purple    gt color: purple
has round surface: True
****************
****************
predicted shape: cylinder    gt shape: cylinder
predicted size: large    gt size: large
predicted color: green    gt color: green
has round surface: True
****************
****************
predicted shape: sphere    gt shape: sphere
predicted size: medium    gt size: medium
predicted color: red    gt color: red
has round surface: True
****************
****************
predicted shape: sphere    gt shape: sphere
predicted si

  1%|▎                                        | 47/7676 [00:00<01:38, 77.17it/s]

...

****************
predicted shape: capsule    gt shape: capsule
predicted size: small    gt size: small
predicted color: yellow    gt color: yellow
has round surface: True
****************
****************
predicted shape: cylinder    gt shape: cylinder
predicted size: medium    gt size: medium
predicted color: purple    gt color: purple
has round surface: True
****************
****************
predicted shape: cone    gt shape: cone
predicted size: medium    gt size: medium
predicted color: purple    gt color: purple
has round surface: True
****************
****************
predicted shape: cylinder    gt shape: cone
predicted size: small    gt size: small
predicted color: red    gt color: purple
has round surface: True
****************
****************
predicted shape: cone    gt shape: cone
predicted size: medium    gt size: medium
predicted color: red    gt color: red
has round surface: True
****************
****************
predicted shape: cone    gt shape: cone
predicted size: la

100%|██████████████████████████████████████▉| 7664/7676 [01:30<00:00, 68.86it/s]

****************
predicted shape: capsule    gt shape: capsule
predicted size: large    gt size: large
predicted color: blue    gt color: blue
has round surface: True
****************
****************
predicted shape: box    gt shape: box
predicted size: medium    gt size: medium
predicted color: blue    gt color: blue
has round surface: True
****************
****************
predicted shape: sphere    gt shape: sphere
predicted size: medium    gt size: medium
predicted color: yellow    gt color: yellow
has round surface: True
****************
****************
predicted shape: cylinder    gt shape: cylinder
predicted size: small    gt size: small
predicted color: blue    gt color: blue
has round surface: True
****************
****************
predicted shape: cone    gt shape: cone
predicted size: small    gt size: small
predicted color: blue    gt color: blue
has round surface: True
****************
****************
predicted shape: cylinder    gt shape: cylinder
predicted size: mediu

100%|███████████████████████████████████████| 7676/7676 [01:30<00:00, 84.55it/s]

****************
predicted shape: cylinder    gt shape: cylinder
predicted size: small    gt size: small
predicted color: red    gt color: red
has round surface: True
****************
****************
predicted shape: cylinder    gt shape: cylinder
predicted size: medium    gt size: medium
predicted color: blue    gt color: blue
has round surface: True
****************
****************
predicted shape: sphere    gt shape: sphere
predicted size: medium    gt size: medium
predicted color: purple    gt color: purple
has round surface: True
****************
****************
predicted shape: cylinder    gt shape: cone
predicted size: medium    gt size: large
predicted color: red    gt color: yellow
has round surface: True
****************
****************
predicted shape: box    gt shape: box
predicted size: large    gt size: large
predicted color: red    gt color: red
has round surface: True
****************
****************
predicted shape: cone    gt shape: cone
predicted size: small    

In [15]:
print("shape prediction accuracy = %f" % (shape_correct / test_size))
print("size prediction accuracy = %f" % (size_correct / test_size))
print("color prediction accuracy = %f" % (color_correct / test_size))

shape prediction accuracy = 0.902423
size prediction accuracy = 0.974857
color prediction accuracy = 0.942809
